In [1]:
# imports
import decimal # for 100% precision fractional operations (doesn't support most math functions like sqrt)
from decimal import Decimal
from bigfloat import * # for high precision float operations (supports most math functions like sqrt, often faster than decimal)
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
import math

In [2]:
# global settings
plt.style.use('dark_background')

In [3]:
# testing bigfloat lib
with precision(200) + RoundTowardZero:
    foo = sqrt(2)

print(foo.__repr__)
print(float(foo))

<bound method BigFloat.__repr__ of BigFloat.exact('1.4142135623730950488016887242096980785696718753769480731766796', precision=200)>
1.4142135623730951


## Data functions

In [5]:
def BigFloat_to_Decimal(x):
    try:
        return (Decimal(e.__str__()) for e in x)
    except TypeError:
        return Decimal(x.__str__())

def Decimal_to_BigFloat(x):
    try:
        return (BigFloat(e.__str__()) for e in x)
    except TypeError:
        return BigFloat(x.__str__())


# like python range, but for decimals
def drange(x, y, jump):
    while x <= y:
        yield float(x)
        x += Decimal(jump)


##  Math functions

### <p style="text-align: center;"> Wald interval </p>

$$(w^-, w^+) = \hat{p}\,\pm\,z\sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$$

In [6]:
def wald_interval(x, n, conflevel = 0.95):
    p = x/n
    var = (p*(1-p))/n
    sd = math.sqrt((p*(1-p))/n)
    z = (norm.ppf((1-conflevel)/2), norm.ppf(1 - ((1-conflevel)/2) ) ) 
    ci = (p + z[0]*sd, p + z[1]*sd)
    return ci


In [7]:


numSamples = 10000
numTrials = 200
probs = drange(Decimal('0.001'), Decimal('0.999'), Decimal('0.001'))
# print(len(list(probs)))

# ci = wald_interval(x=20,n=40)

# print(list(range(1, numSamples)))



calculating currently on prob=0.001
calculating currently on prob=0.002
calculating currently on prob=0.003
calculating currently on prob=0.01
calculating currently on prob=0.02
calculating currently on prob=0.05


KeyboardInterrupt: 

### <p style="text-align: center;"> Wilson score interval </p>

$$(w^-, w^+) = \frac{p + z^2/2n \pm z\sqrt{p(1-p)/n + z^2/4n^2}}{1+z^2/n}$$

### <p style="text-align: center;"> Wilson score interval (continuity-corrected) </p>

$$w_{cc}^- = \frac{2np + z^2 - (z\sqrt{z^2 - 1/n + 4np(1-p) + (4p-2)} + 1)}{2(n+z^2)}$$


$$w_{cc}^+ = \frac{2np + z^2 + (z\sqrt{z^2 - 1/n + 4np(1-p) - (4p-2)} + 1)}{2(n+z^2)}$$

or, simplified:

$$e = 2np + z^2;\,\,\, f = z^2 - 1/n + 4np(1-p);\,\,\, g = (4p - 2);\,\,\, h = 2(n+z^2)$$

$$w_{cc}^- = \frac{e - (z\sqrt{f+g} + 1)}{h}$$

$$w_{cc}^+ = \frac{e + (z\sqrt{f-g} + 1)}{h}$$

The best performance for values of $p$ from $2\cdot10^{-5}$ to $2\cdot10^{-4}$ and values of $n$ from $20000$ to $40000$ showed 2 methods:

Wilson Score Interval and Wilson score interval (continuity-semi-corrected). The latter is just the arithmetic mean between uncorrected Wilson Score Interval and the continuity-corrected.

Lets use them.

## <p style="text-align: center;">CI for the difference between two proportions</p>

### <p style="text-align: center;">Z test (Unpooled)</p>

$$ z_{t} = \frac{\hat{p}_T - \hat{p}_C - \delta}{\sqrt{\frac{\hat{p}_T (1 - \hat{p}_T)}{n_T} + \frac{\hat{p}_C (1 - \hat{p}_C)}{n_C}}} $$

lower limit is found by solving $z_{t} = |z_{\alpha}|$ 

and the upper limit is found by solving $z_{t} = -|z_{\alpha}|$


$$ (\delta^-, \delta^+) = \hat{p}_T - \hat{p}_C \pm z_{\alpha}\sqrt{\frac{\hat{p}_T (1 - \hat{p}_T)}{n_T} + \frac{\hat{p}_C (1 - \hat{p}_C)}{n_C}} $$

<br>

### <p style="text-align: center;">Z test (Pooled)</p>

$$ z_{t} = \frac{\hat{p}_T - \hat{p}_C - \delta}{\sqrt{\bar{p}(1-\bar{p})(\frac{1}{n_T}+\frac{1}{n_C})}} $$

$$\bar{p} = \frac{n_T\hat{p}_T + n_C\hat{p}_C}{n_T + n_C}$$

lower limit is found by solving $z_{t} = |z_{\alpha}|$ 

and the upper limit is found by solving $z_{t} = -|z_{\alpha}|$


$$ (\delta^-, \delta^+) = \hat{p}_T - \hat{p}_C \pm z_{\alpha}\sqrt{\bar{p}(1-\bar{p})(\frac{1}{n_T}+\frac{1}{n_C})} $$

